In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/home/vmeshchaninov/DiffusionTextGeneration-cond-ca/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
from torch.nn.functional import cross_entropy
from transformers import AutoTokenizer, T5Model, T5ForConditionalGeneration

In [19]:
t5 = T5ForConditionalGeneration.from_pretrained("t5-base")

In [20]:
tokenizer = AutoTokenizer.from_pretrained("t5-large")

/home/vmeshchaninov/anaconda3/envs/env/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
text_cond = "Unlike ordinary beam search, constrained beam search allows us to exert control over the " \
            "output of text generation. This is useful because we sometimes " \
            "know exactly what we want inside the output. For example, in a Neural Machine Translation task"
        
text_uncond = "we might know which words must be included in the final translation with a dictionary lookup. " \
              "Sometimes, generation outputs that are almost equally possible to a language model might not be equally " \
              "desirable for the end-user due to the particular context. Both of these situations could be solved by " \
              "allowing the users to tell the model which words must be included in the end output."

In [21]:
input_ids = tokenizer(text_cond, return_tensors="pt").input_ids
labels = tokenizer(text_uncond, return_tensors="pt").input_ids
labels_s = t5._shift_right(labels)

outputs = t5(input_ids=input_ids, decoder_input_ids=labels_s)

In [22]:
cross_entropy(outputs.logits.reshape(-1, 32128), labels.reshape(-1))

tensor(3.6221, grad_fn=<NllLossBackward0>)

In [29]:
tokenizer.batch_decode(labels_s, skip_special_tokens=True)

['we might know which words must be included in the final translation with a dictionary lookup. Sometimes, generation outputs that are almost equally possible to a language model might not be equally desirable for the end-user due to the particular context. Both of these situations could be solved by allowing the users to tell the model which words must be included in the end output.']

In [25]:
tokenizer.batch_decode(outputs.logits.argmax(dim=-1), skip_special_tokens=True)

['want want exactly words we be translated in the output output. a particular.up., we of is are are not identical important are producea given are are be be the useful. different user useruser. to the large constraints. In methods these cases are be solved by cona us user to specify the system what words to be included. the output-.']

In [24]:
n = 10
proposed = torch.argsort(outputs.logits, dim=-1, descending=True)[:, :, :n].reshape(-1, n)
true = labels.reshape(-1)

for i in range(true.shape[0]):
    j = -1
    for j_ in range(n):
        if true[i] == proposed[i][j_]:
            j = j_
    s = f"true: {true[i]}, {' ' * (5 - len(str(true[i].item())))} pr_ind: {j} {' ' * (5 - len(str(j)))} proposed: {proposed[i].data.numpy()}"
    print(s)

true: 62,     pr_ind: -1     proposed: [32099     3     6     5     1     7    18 32086    11 32092]
true: 429,    pr_ind: 1      proposed: [241 429  54 214 164 174 557 133  43 169]
true: 214,    pr_ind: 1      proposed: [  241   214   174 11610   169    43    36  1672  6634  2204]
true: 84,     pr_ind: 6      proposed: [ 1776   125    24 11185     8   149    84   131     3   213]
true: 1234,   pr_ind: 0      proposed: [ 1234  7142 16513  1499  1448   294  3785     3  1467  1612]
true: 398,    pr_ind: -1     proposed: [  62   12   33  225   42   16   11   56    8 1776]
true: 36,     pr_ind: 0      proposed: [  36 2385 3480  281   43  369 1588 4093 2367 1130]
true: 1285,   pr_ind: 1      proposed: [15459  1285     3   261  6126  3911 21527  2546    16 23734]
true: 16,     pr_ind: 0      proposed: [  16    5 1096   11  441   42  139   12    6   21]
true: 8,      pr_ind: 0      proposed: [   8    3 3911   46   69  284 1499 3785  125 7314]
true: 804,    pr_ind: 4      proposed: [ 3911  378

In [33]:
tokenizer.decode(18)

'-'

In [34]:
name = "EleutherAI/gpt-neo-2.7B"

In [35]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

In [36]:
gpt = GPTNeoForCausalLM.from_pretrained(name)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(name)

In [38]:
input_ids = tokenizer(text_cond, return_tensors="pt").input_ids

outputs = gpt(input_ids=input_ids, labels=input_ids)

In [39]:
cross_entropy(outputs.logits.reshape(-1, 50257)[:-1], input_ids.reshape(-1)[1:])

tensor(3.2333, grad_fn=<NllLossBackward0>)

In [44]:
n = 10
proposed = torch.argsort(outputs.logits.reshape(-1, 50257)[:-1], dim=-1, descending=True)[:, :n].reshape(-1, n)
true = input_ids.reshape(-1)[1:]

for i in range(true.shape[0]):
    j = -1
    for j_ in range(n):
        if true[i] == proposed[i][j_]:
            j = j_
    s = f"true: {true[i]}, {' ' * (5 - len(str(true[i].item())))} pr_ind: {j} {' ' * (5 - len(str(j)))} proposed: {proposed[i].data.numpy()}"
    print(s)

true: 8850,   pr_ind: -1     proposed: [262 867 257 428 749 584 883 674 616 287]
true: 15584,  pr_ind: -1     proposed: [ 661 1692 5384 4290 1450 3788   11 3644 3925 5006]
true: 2989,   pr_ind: -1     proposed: [15464    12 19702 14583  6626 21103 43813 23610  3777  5289]
true: 11,     pr_ind: 0      proposed: [   11 16113  5050  3341  7605  2761   290   393  9021 10064]
true: 31070,  pr_ind: -1     proposed: [  262   674   356   257   428   543   810 15584   287   281]
true: 15584,  pr_ind: 1      proposed: [ 2989 15584 23989    12  1957  4738  4696   416   850  1366]
true: 2989,   pr_ind: 0      proposed: [ 2989 15455    12 10342 15464 12947   198   357 23989 21976]
true: 3578,   pr_ind: 4      proposed: [  357   318   460   468  3578  7228   685  3544 12031    11]
true: 514,    pr_ind: 4      proposed: [  262   257   284   329   514   530 15584   345   281   517]
true: 284,    pr_ind: 0      proposed: [ 284  407  198   11  262  257 2989  357  691  517]
true: 17596,  pr_ind: -1     p

In [45]:
from transformers import AutoConfig

In [47]:
AutoConfig.from_pretrained("t5-base")

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "pre